In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
from etrago import Etrago
from etrago import tools
from etrago.tools.plot import (
    coloring, 
    make_legend_circles_for, 
    make_handler_map_to_scale_circles_as_in,
    plot_grid,
    calc_storage_expansion_per_bus,
)

colors = coloring

Import networks

In [ ]:
# etrago_b = Etrago(csv_folder_name= "network_raw_beforeAdjustNetwork")
# network_b = etrago_b.network.copy()

etrago = Etrago(csv_folder_name= "network_raw_pfp")
network = etrago.network.copy()

# etrago_a = Etrago(csv_folder_name= "network_aggregated")
# network_a = etrago_a.network.copy()

In [ ]:
print(network)

# Buses

In [ ]:
buses_DE = network.buses.loc[network.buses.country == 'DE']
buses_abroad = network.buses.loc[network.buses.country != 'DE']

In [ ]:
# print(network.buses.carrier.unique())
print(network.buses.carrier.value_counts())

## H2 buses

In [ ]:
H2_over_buses = network.buses[(network.buses.carrier == "H2_grid")]
# H2_over_buses

In [ ]:
H2_under_buses = network.buses[(network.buses.carrier == "H2_saltcavern")]
# H2_under_buses

# Loads

In [ ]:
# print(network.loads.carrier.unique())
print(network.loads.carrier.value_counts())

## H2 loads

In [ ]:
rel_h2_loads_carriers = ['H2_for_industry', 'H2_hgv_load']
# rel_h2_loads_carriers


In [ ]:
H2_for_industry_load_DE = network.loads[(network.loads.carrier == 'H2_for_industry') & (network.loads.bus.isin(buses_DE.index))]
H2_for_industry_load_DE_t = network.loads_t.p_set[H2_for_industry_load_DE.index.to_list()]

print(f"Total H2_for_industry_load_DE (after adjust_network) {H2_for_industry_load_DE_t.sum(axis=1).sum()/1e6:.2f} TWh")
print(H2_for_industry_load_DE_t.sum(axis=1).sum()/1e6)
# H2_for_industry_load_DE
# H2_for_industry_load_DE_t

In [ ]:
H2_for_industry_load_abroad = network.loads[(network.loads.carrier == 'H2_for_industry') & network.loads.bus.isin(buses_abroad.index)]
H2_for_industry_load_abroad[["bus", "p_set"]]
# H2_for_industry_load_abroad

In [ ]:
H2_hgv_load = network.loads[network.loads.carrier == 'H2_hgv_load']
H2_hgv_load_t = network.loads_t.p_set[H2_hgv_load.index.to_list()]
# H2_hgv_load
# H2_hgv_load_t

In [ ]:
# Fig resampled

fig, ax = plt.subplots(figsize = (10,5))

(H2_hgv_load_t.sum(axis=1).resample('20H').mean()/1e3).plot(
    ax=ax,
    label = "H2_hgv_load", 
    legend=True, 
)

(H2_for_industry_load_DE_t.sum(axis=1).resample('20H').mean()/1e3).plot( #
    ax=ax,
    label = "H2_for_industry_load_DE", 
    legend=True, 
)

# (H2_for_industry_load_abroad/1e3).plot(
#     ax=ax,
#     label = "H2_for_industry_load_abroad", 
#     legend=True, 
# )

ax.set_ylabel('GW')
ax.set_title('H2 loads')
fig.show()

In [ ]:
# Fig NOT resampled

fig, ax = plt.subplots(figsize = (10,5))

(H2_hgv_load_t.sum(axis=1)/1e3).plot(
    ax=ax,
    label = "H2_hgv_load", 
    legend=True, 
)

(H2_for_industry_load_DE_t.sum(axis=1)/1e3).plot(
    ax=ax,
    label = "H2_for_industry_load_DE", 
    legend=True, 
)

# (H2_for_industry_load_abroad/1e3).plot(
#     ax=ax,
#     label = "H2_for_industry_load_abroad", 
#     legend=True, 
# )

ax.set_ylabel('GW')
ax.set_title('H2 loads')
fig.show()

## CH4 loads

In [ ]:
rel_ch4_loads_carriers = ['rural_gas_boiler', 'CH4_for_industry', 'CH4']

In [ ]:
rural_gas_boiler_load = network.loads[(network.loads.carrier == 'rural_gas_boiler')]
rural_gas_boiler_load_t = network.loads_t.p_set[rural_gas_boiler_load.index.to_list()]

print(f"Total rural_gas_boiler_load (Germany) {rural_gas_boiler_load_t.sum(axis=1).sum()/1e6:.2f} TWh")
# rural_gas_boiler_load
# rural_gas_boiler_load_t

In [ ]:
CH4_for_industry_load_DE = network.loads[(network.loads.carrier == 'CH4_for_industry')]
CH4_for_industry_load_DE_t = network.loads_t.p_set[CH4_for_industry_load_DE.index.to_list()]

print(f"Total CH4_for_industry_load_DE (after adjust_network): {CH4_for_industry_load_DE_t.sum(axis=1).sum()/1e6:.2f} TWh")
# CH4_for_industry_load_DE
# CH4_for_industry_load_DE_t

In [ ]:
CH4_abroad_load = network.loads[(network.loads.carrier == 'CH4')]
CH4_abroad_load_t = network.loads_t.p_set[CH4_abroad_load.index.to_list()]

print(CH4_abroad_load_t.sum())
# CH4_abroad_load
# CH4_abroad_load_t

In [ ]:
CH4_abroad_load_t_corrected = network.loads_t.p_set[CH4_abroad_load.index.to_list()] *8760

print(CH4_abroad_load_t_corrected.sum())
# CH4_abroad_load
# CH4_abroad_load_t

In [ ]:
fig, ax = plt.subplots(figsize = (10,5))

(rural_gas_boiler_load_t.sum(axis=1).resample('20H').mean()/1e3).plot(
    ax=ax,
    label = "rural_gas_boiler_load_t", 
    legend=True, 
)

(CH4_for_industry_load_DE_t.sum(axis=1).resample('20H').mean()/1e3).plot( #
    ax=ax,
    label = "CH4_for_industry_load_DE", 
    legend=True, 
)

(CH4_abroad_load_t.sum(axis=1).resample('20H').mean()/1e3).plot( #
    ax=ax,
    label = "CH4_abroad_load", 
    legend=True, 
)

(CH4_abroad_load_t_corrected.sum(axis=1).resample('20H').mean()/1e3).plot( #
    ax=ax,
    label = "CH4_abroad_load_corrected", 
    legend=True, 
)

ax.set_ylabel('GW')
ax.set_title('CH4 loads')
fig.show()

# Stores

In [ ]:
# print(network.stores.carrier.unique())
print(network.stores.carrier.value_counts())

## H2 underground stores

In [ ]:
H2_underground_stores = network.stores[(network.stores.carrier == "H2_underground")]
# H2_underground_stores

In [ ]:
H2_saltcavern_potential =  H2_underground_stores.e_nom_max.sum()

print(f'H2_underground total storage capacity potential: {H2_saltcavern_potential/1e3:.2f} GWh')

In [ ]:
h2_saltcavern_store_cap =  H2_underground_stores.set_index("bus")["e_nom_max"]

network_f = network.copy()
network_f.buses = network_f.buses[(network_f.buses.carrier.isin(["H2_saltcavern", "H2_grid"]))]

scaling = 1/1e3
bus_unit = "GWh"

bus_scaling = 1/6e3
bus_sizes = h2_saltcavern_store_cap * scaling


handles = make_legend_circles_for([
        # bus_sizes.min()*bus_scaling, 
        bus_sizes.max()*bus_scaling
    ], 
    scale=1, 
    facecolor=colors()["H2_underground"]
)

labels = [#"min", "max"]
    ("{} " + bus_unit).format(s)
    for s in (
        # round(bus_sizes.min()),
        round(bus_sizes.max()),
    )
]

fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()})#, figsize=(12,8)
network_f.plot(
    bus_sizes=bus_sizes*bus_scaling,
    margin=0.1,
    ax=ax,
    bus_colors=colors()["H2_underground"],
)
l2 = ax.legend(
    handles,
    labels,
    loc='lower left',
    # bbox_to_anchor=(0.01, 1.01),
    labelspacing=1.0,
    framealpha=1.0,
    # title=bus_legend,
    handler_map=make_handler_map_to_scale_circles_as_in(ax),
)
ax.add_artist(l2)

ax.set_title("H2 saltcavern storage potentials for eGon2035")
fig.show()

## CH4 stores

In [ ]:
CH4_stores = network.stores[(network.stores.carrier == "CH4")]
# CH4_stores

In [ ]:
CH4_stores_DE = network.stores[(network.stores.carrier == "CH4") & (network.stores.bus.isin(buses_DE.index))]
# CH4_stores_DE

In [ ]:
CH4_stores_cap_DE =  CH4_stores_DE.e_nom.sum()

print(f'CH4 total storage capacity in Germany: {CH4_stores_cap_DE/1e6:.2f} TWh')

In [ ]:
ch4_store_cap = CH4_stores_DE.set_index("bus")["e_nom"]

unit_scaling = 1/1e6
bus_unit = "TWh"

bus_scaling = 1/2e2
bus_sizes = ch4_store_cap * unit_scaling


handles = make_legend_circles_for([
        # bus_sizes.min()*bus_scaling, 
        bus_sizes.max()*bus_scaling
    ], 
    scale=1, 
    facecolor="blue",
)

labels = [#"min", "max"]
    ("{} " + bus_unit).format(s)
    for s in (
        # round(bus_sizes.min()),
        round(bus_sizes.max()),
    )
]

network_f = network.copy()
network_f.buses = network_f.buses[
    (network_f.buses.carrier == "CH4")
    & (network_f.buses.index.isin(buses_DE.index))
]

fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()})
network_f.plot(
    bus_sizes=bus_sizes * bus_scaling,
    bus_colors="blue",
    line_widths=0,
    link_widths=0,
    margin=0.1,
    ax=ax,
    title="CH4 stores",
)
l2 = ax.legend(
    handles,
    labels,
    loc='lower left',
    # bbox_to_anchor=(0.01, 1.01),
    labelspacing=1.0,
    framealpha=1.0,
    # title=bus_legend,
    handler_map=make_handler_map_to_scale_circles_as_in(ax),
)
ax.add_artist(l2)


fig.show()

In [ ]:
ch4_store_cap = CH4_stores.set_index("bus")["e_nom"]

unit_scaling = 1/1e6
bus_unit = "TWh"

bus_scaling = 1/1e2
bus_sizes = ch4_store_cap * unit_scaling


handles = make_legend_circles_for([
        # bus_sizes.min()*bus_scaling, 
        bus_sizes.max()*bus_scaling
    ], 
    scale=1, 
    facecolor="blue",
)

labels = [#"min", "max"]
    ("{} " + bus_unit).format(s)
    for s in (
        # round(bus_sizes.min()),
        round(bus_sizes.max()),
    )
]

network_f = network.copy()
network_f.buses = network_f.buses[
    (network_f.buses.carrier == "CH4")
    & (network_f.buses.country != "RU")
]

fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()})
network_f.plot(
    bus_sizes=bus_sizes * bus_scaling,
    bus_colors="blue",
    line_widths=0,
    link_widths=0,
    margin=0.1,
    ax=ax,
    title="CH4 stores",
)
l2 = ax.legend(
    handles,
    labels,
    loc='lower left',
    # bbox_to_anchor=(0.01, 1.01),
    labelspacing=1.0,
    framealpha=1.0,
    # title=bus_legend,
    handler_map=make_handler_map_to_scale_circles_as_in(ax),
)
ax.add_artist(l2)

fig.show()

In [ ]:
CH4_stores_a = network.stores[(network.stores.carrier == "CH4") & (network.stores.bus.isin(buses_abroad.index))]
print(buses_abroad[buses_abroad.carrier =="CH4"]["country"])
CH4_stores_a.set_index("bus")["e_nom"]/1e6

# Generators (CH4)

In [ ]:
# print(network.generators.carrier.unique())
print(network.generators.carrier.value_counts())

In [ ]:
ch4_carriers = ['CH4', 'CH4_NG', 'CH4_biogas']
ch4_carriers_DE = ['CH4_NG', 'CH4_biogas']

ch4_gen = network.generators[(network.generators.carrier.isin(ch4_carriers))]
ch4_gen_DE = network.generators[(network.generators.carrier.isin(ch4_carriers_DE))]

In [ ]:
ch4_gen_abroad = network.generators[(network.generators.carrier == "CH4")]
print(buses_abroad[buses_abroad.carrier =="CH4"]["country"])
# print(ch4_gen_abroad[["bus","p_nom"]])
print(ch4_gen_abroad.set_index("bus")["e_nom_max"]/1e6)
# ch4_gen_abroad

In [ ]:
ch4_gen_biogas = network.generators[(network.generators.carrier == "CH4_biogas")]

print(f'Total biogas production capacity in Germany: {ch4_gen_biogas["p_nom"].sum()/1e3:.2f} GW')
# ch4_gen_biogas

In [ ]:
ch4_gen_NG = network.generators[(network.generators.carrier == "CH4_NG")]

print(f'Total natural gas production capacity in Germany: {ch4_gen_NG["p_nom"].sum()/1e3:.2f} GW')
# ch4_gen_NG

# Links

In [ ]:
# print(network.links.carrier.unique())
print(network.links.carrier.value_counts())

## CH4 Links

In [ ]:
rel_ch4_links_carriers = network.links.loc[network.links.bus0.isin(network.buses.loc[network.buses.carrier == 'CH4'].index)].carrier.unique()
rel_ch4_links_carriers

### CH4 grid

In [ ]:
CH4_pipelines = network.links[network.links.carrier == "CH4"]
# CH4_pipelines

### Gas turbines (OCGT)

In [ ]:
gas_turbines = network.links[network.links.carrier == "OCGT"]
gas_turbines_DE = gas_turbines[gas_turbines.country == "DE"]
# gas_turbines_DE
print(f'Total OCGT capacity (Germany): {gas_turbines_DE["p_nom"].sum()/1e3:.2f} GW')

In [ ]:
gas_turbines_abroad = gas_turbines[gas_turbines.country != "DE"]
print(gas_turbines_abroad.set_index("country")["p_nom"]*0.42)

## H2 links

In [ ]:
rel_h2_links_carriers = network.links.loc[network.links.bus1.isin(network.buses.loc[(network.buses.carrier == 'H2_grid')|(network.buses.carrier == 'H2_salcavern')].index)].carrier.unique()
rel_h2_links_carriers